In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate

pandas: pd로 별칭 지정한 패키지로, 데이터 조작과 분석에 유용한 기능을 제공하는 라이브러리입니다. 주로 표 형태의 데이터를 다룰 때 사용되며, 데이터프레임(DataFrame)이라는 자료구조를 제공합니다.

tensorflow: tf로 별칭 지정한 딥러닝 프레임워크로, 기계 학습과 딥러닝 모델을 구축하고 학습시키는 데 사용됩니다. TensorFlow는 텐서(Tensor)라는 다차원 배열을 다루는 라이브러리로, 그래프 기반의 연산을 수행합니다.

tensorflow.keras: TensorFlow에서 제공하는 고수준 딥러닝 API로, Keras API를 TensorFlow에 통합한 것입니다. 딥러닝 모델을 쉽게 구축하고 학습시킬 수 있는 편리한 인터페이스를 제공합니다.

preprocessing: tensorflow.keras.preprocessing 모듈은 데이터 전처리에 사용됩니다. 주로 텍스트, 이미지, 시계열 데이터 등을 신경망 모델에 입력하기 전에 사전 처리하는 데 사용됩니다. 예를 들어, 텍스트 데이터의 토큰화, 정수 인코딩, 패딩 등의 작업을 처리할 수 있습니다.

tensorflow.keras.models: TensorFlow에서 제공하는 모델 구성을 위한 모듈입니다. 주요 모델 구성 요소인 층(layer)들을 조합하여 모델을 생성하고 관리하는 기능을 제공합니다.

tensorflow.keras.layers: TensorFlow에서 제공하는 다양한 종류의 층(layer)들을 포함하는 모듈입니다. 예를 들어, Dense 층은 완전 연결 신경망(fully connected neural network)을 구성하는 데 사용되며, Conv1D 층은 1차원 합성곱 신경망(convolutional neural network)을 구성하는 데 사용됩니다. Embedding 층은 텍스트나 범주형 데이터를 처리하는 데 사용됩니다.

Input 레이어는 신경망 모델의 입력 데이터를 정의하는 역할을 합니다. 신경망은 데이터를 입력받아 여러 계층을 거쳐 출력을 생성하는 구조로 이루어져 있습니다. Input 레이어는 이러한 입력 데이터를 정의하고, 모델의 첫 번째 레이어로 사용됩니다.
여기서 shape은 입력 데이터의 형태를 정의하는 매개변수입니다. 입력 데이터의 모양(shape)은 일반적으로 (batch_size, input_shape)의 형태를 갖습니다. input_shape은 데이터의 차원(dimension)을 나타내는 튜플이며, 예를 들어 이미지의 경우 (height, width, channels)와 같은 형태가 될 수 있습니다.

In [2]:
# 데이터 읽어오기
train_file = "total_train_data.csv"

In [3]:
data = pd.read_csv(train_file, delimiter = ',')
queries = data['query'].tolist()
intents = data['intent'].tolist()

In [4]:
import os, sys    # 다른 폴더에 있는것들 가져오기 위함

module_paths = [
    'C:\\Users\\ledu2\\NLP\\chatbot_study\\utils',
    'C:\\Users\\ledu2\\NLP\\chatbot_study\\config'
]

for path in module_paths:
    abs_path = os.path.abspath(path)
    if abs_path not in sys.path:
        sys.path.append(abs_path)

In [29]:
from Preprocess import Preprocess
p = Preprocess(word2index_dic = '../../train_tools/dict/chatbot_dict.bin', 
              userdic = '../../utils/user_dic.tsv')

In [6]:
# 데이터 시퀀스 생성
sequences = []
for sentence in queries:
    pos = p.pos(sentence)
    keywords = p.get_keywords(pos, without_tag = True)
    seq = p.get_wordidx_sequence(keywords)
    sequences.append(seq)

In [7]:
# 단어 인덱스 시퀀스 벡터 생성
# 단어 시퀀스 백터 크기
from GlobalParams import MAX_SEQ_LEN
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen = MAX_SEQ_LEN,
                                                  padding = 'post')

In [9]:
# 학습용, 검증용, 테스트용 데이터셋 생성
# 학습셋:검증셋:테스트셋 = 7:2:1
ds = tf.data.Dataset.from_tensor_slices((padded_seqs, intents))
ds = ds.shuffle(len(queries))

train_size = int(len(padded_seqs) * 0.7)
val_size = int(len(padded_seqs) * 0.2)
test_size = int(len(padded_seqs) * 0.1)

train_ds = ds.take(train_size).batch(20)
val_ds = ds.skip(train_size).take(val_size).batch(20)
test_ds = ds.skip(train_size + val_size).take(test_size).batch(20)


tf.data.Dataset.from_tensor_slices((padded_seqs, intents)):

padded_seqs: 패딩된 시퀀스 데이터를 담고 있는 텐서입니다. 시퀀스 데이터는 문장이나 문서를 숫자로 변환한 것을 말합니다.
intents: 각 시퀀스 데이터에 대응하는 의도(라벨) 정보를 담고 있는 텐서입니다.
from_tensor_slices() 메서드를 사용하여 텐서를 슬라이스하고 데이터셋을 생성합니다. 이렇게 하면 시퀀스 데이터와 의도 정보가 쌍으로 매칭된 데이터셋이 생성됩니다.
ds.shuffle(len(queries)):

ds: 생성한 데이터셋 객체입니다.
shuffle(len(queries)) 메서드를 사용하여 데이터셋을 섞습니다. len(queries)는 데이터셋의 총 개수를 의미합니다. 데이터셋을 섞는 것은 모델이 데이터의 순서를 기억하지 못하도록 하고, 더 좋은 학습 결과를 얻기 위해 데이터를 무작위로 배치하는데 도움을 줍니다.
이 코드들을 실행하면 입력 데이터를 TensorFlow의 데이터셋으로 구성하고, 데이터셋을 섞어서 모델에 사용할 준비를 완료합니다. 이후에는 데이터셋을 배치하거나 반복하여 모델에 입력으로 제공할 수 있습니다.

In [13]:
# 하이퍼파라미터 설정 
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(p.word_index)+1    # 전체 단어 수

In [14]:
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE, input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

In [20]:
conv1 = Conv1D(
        filters = 128,
        kernel_size = 3,
        padding = 'valid',    # valid 패딩을 하면 결과는 항상 입력보다 작아짐
        activation = tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)

In [21]:
conv2 = Conv1D(
        filters = 128,
        kernel_size = 4,
        padding = 'valid',
        activation = tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)

In [22]:
conv3 = Conv1D(
        filters = 128,
        kernel_size = 5,
        padding = 'valid',
        activation = tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)

In [23]:
# 3, 4, 5-gram 이후 합치기
concat = concatenate([pool1, pool2, pool3])

In [24]:
hidden = Dense(128, activation = tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(5, name='logits')(dropout_hidden)
predictions = Dense(5, activation = tf.nn.softmax)(logits)

In [25]:
# 모델 생성
#이진 분류의 문제에서는 주로 binary_crossentropy를 사용하고,
#다중 분류에서는 categorical_crossentropy, 
#수치형 변수를 예측할 때는 mean_squared_error를 주로 사용
model = Model(inputs = input_layer, outputs = predictions)
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [26]:
# 모델 학습
# verbose 0 은 출력하지 않고, 1은 자세히, 2는 함축적인 정보

model.fit(train_ds, validation_data = val_ds, epochs = EPOCH, verbose = 1)

Epoch 1/5
3698/3698 [==============================] - 173s 47ms/step - loss: 0.0482 - accuracy: 0.9854 - val_loss: 0.0157 - val_accuracy: 0.9934
Epoch 2/5
3698/3698 [==============================] - 189s 51ms/step - loss: 0.0156 - accuracy: 0.9946 - val_loss: 0.0089 - val_accuracy: 0.9957
Epoch 3/5
3698/3698 [==============================] - 195s 53ms/step - loss: 0.0120 - accuracy: 0.9952 - val_loss: 0.0076 - val_accuracy: 0.9969
Epoch 4/5
3698/3698 [==============================] - 201s 54ms/step - loss: 0.0096 - accuracy: 0.9961 - val_loss: 0.0071 - val_accuracy: 0.9967
Epoch 5/5
3698/3698 [==============================] - 200s 54ms/step - loss: 0.0073 - accuracy: 0.9968 - val_loss: 0.0052 - val_accuracy: 0.9975


In [27]:
# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose = 1)
print('Accuracy: %f' % (accuracy  * 100))
print('loss: %f' % (loss))

529/529 [==============================] - 1s 2ms/step - loss: 0.0065 - accuracy: 0.9975
Accuracy: 99.753904
loss: 0.006529


In [28]:
# 모델 저장
model.save('intent_model.h5')